* https://uniswapv3book.com/index.html
* https://medium.com/@chaisomsri96/defi-math-uniswap-v3-concentrated-liquidity-bd87686b3ecf
* https://liobaheimbach.github.io/assets/pdf/Papers/Risks_and_Returns_of_Uniswap_V3_Liquidity_Providers.pdf
* https://docs.uniswap.org/sdk/v3/guides/advanced/active-liquidity
* https://blog.uniswap.org/uniswap-v3-math-primer

In [ ]:
from uniswappy import *
import pandas as pd
import copy
import matplotlib.pyplot as plt

user_nm = MockAddress().apply()
eth_amount = 10000
tkn_amount = 100000

fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [ ]:
def get_current_liquidity(lp, user, positions):
    _id = UniV3Utils.getPositionKey(user, lwr_tick, upr_tick)
    if(_id in positions):
        return positions[_id].liquidity
    else:
        return 0
        
def update_graph(lp, user, tkn, positions, graph):
    curr = get_current_liquidity(lp, user, positions)
    delta_liq = get_current_liquidity(lp, user, lp.positions) - curr
    #print(f'curr {curr}')
    if not user in graph:
        graph[user_add] = {} 
        graph[user_add]['price'] = lp.get_price(tkn)
        graph[user_add]['lwr_tick'] = lwr_tick
        graph[user_add]['upr_tick'] = upr_tick
        graph[user_add]['delta_liq'] = delta_liq
    else:  
        graph[user_add]['price'] = lp.get_price(tkn)
        graph[user_add]['lwr_tick'] = lwr_tick
        graph[user_add]['upr_tick'] = upr_tick
        graph[user_add]['delta_liq'] += delta_liq        


In [ ]:
n_steps = 2500
start_price = eth_amount/tkn_amount
mu = 0.1; sigma = 0.5
n_paths = 1

b = BrownianModel(start_price)
p_arr = b.gen_gbms(mu, sigma, n_steps, n_paths)
exp_p_arr = np.median(p_arr, axis = 1)

accounts = MockAddress().apply(250)

In [ ]:
x_val = np.arange(0,len(p_arr))
fig, (USD_ax) = plt.subplots(nrows=1, sharex=False, sharey=False, figsize=(18, 5))
USD_ax.plot(x_val[1:], p_arr[1:], color = 'r',linestyle = 'dashdot', label='initial invest') 
USD_ax.set_title(f'Price Chart (ETH/TKN)', fontsize=20)
USD_ax.set_ylabel('Price (USD)', size=20)
USD_ax.set_xlabel('Date', size=20)

In [ ]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", address="0x011")

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, tkn_amount)
lp.summary()

df_v2 = pd.DataFrame(columns = ['price', 'liq'])

In [ ]:
arb = CorrectReserves(lp, x0 = 1/exp_p_arr[0])
for k in range(1, n_steps):
    p = 1/exp_p_arr[k]
    arb.apply(p)
    
    select_tkn = EventSelectionModel().bi_select(0.5)
    rnd_swap_amt = TokenDeltaModel(30).delta()
    rnd_add_amt = TokenDeltaModel(30).delta()

    user_add = random.choice(accounts)
    user_swap = random.choice(accounts)

    # Random trade volume  
    if(select_tkn == 0):
        AddLiquidity().apply(lp, eth, user_add, rnd_add_amt)
        df_v2 = pd.concat([df_v2,pd.DataFrame.from_dict({'price': [lp.get_price(tkn)], 'liq': [lp.last_liquidity_deposit]})])
        out = Swap().apply(lp, eth, user_nm, rnd_swap_amt) 
    else:
        AddLiquidity().apply(lp, tkn, user_add, p*rnd_add_amt)
        df_v2 = pd.concat([df_v2,pd.DataFrame.from_dict({'price': [lp.get_price(tkn)], 'liq': [lp.last_liquidity_deposit]})])
        out = Swap().apply(lp, tkn, user_nm,  p*rnd_swap_amt)  
    
    #print(f'Market: {exp_p_arr[k]}, LP: {lp.get_price(tkn)}')

print('')
lp.summary()

In [ ]:
df_v2.sort_values(by=['price'], inplace=True)
df_v2.reset_index(drop=True,inplace=True)
df_v2['price'] = np.round(df_v2['price'].values, 3)
freq_df_v2 = df_v2.groupby('price').agg({'liq': 'sum'})

In [ ]:
eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, tkn_amount, lwr_tick, upr_tick)
lp.summary()

v3_positions = lp.positions.copy()
v3_graph = {}

In [ ]:
arb = CorrectReserves(lp, x0 = 1/exp_p_arr[0])
for k in range(1, n_steps):
    p = 1/exp_p_arr[k]
    arb.apply(p, lwr_tick, upr_tick)
    
    select_tkn = EventSelectionModel().bi_select(0.5)
    rnd_swap_amt = TokenDeltaModel(30).delta()
    rnd_add_amt = TokenDeltaModel(30).delta()
    user_add = random.choice(accounts)
    user_swap = random.choice(accounts)
    v3_positions = copy.deepcopy(lp.positions)
    if(select_tkn == 0): 
        AddLiquidity().apply(lp, eth, user_add, rnd_add_amt, lwr_tick, upr_tick)
        update_graph(lp, user_add, tkn, v3_positions, v3_graph)
        out = Swap().apply(lp, eth, user_swap, rnd_swap_amt) 
    else:
        AddLiquidity().apply(lp, tkn, user_add, p*rnd_add_amt, lwr_tick, upr_tick)
        update_graph(lp, user_add, tkn, v3_positions, v3_graph)
        out = Swap().apply(lp, tkn, user_swap,  p*rnd_swap_amt) 

    #print(f'Market: {exp_p_arr[k]}, LP: {lp.get_price(tkn)}')

print('')
lp.summary()

In [ ]:
    def get_tick_price(self):  
               
        Q96 = 2**96
        sqrtPriceX96 = lp.slot0.sqrtPriceX96
        return math.floor(math.log((sqrtPriceX96/Q96)**2)/math.log(1.0001))     

In [ ]:
lp.get_tick_price()

In [ ]:
lp.ticks

In [ ]:
UniV3Utils.getPositionKey(accounts[0], lwr_tick, upr_tick)

In [ ]:
lp.positions

In [ ]:
#for pos in v3_graph:
#    print(f"price {v3_graph[pos]['price']}, liq: {v3_graph[pos]['delta_liq']/10**18}")

In [ ]:
for k, pos in enumerate(v3_graph):
    row = np.empty(2)
    row[0] = v3_graph[pos]['price']
    row[1] = v3_graph[pos]['delta_liq']/10**18
    res = row if k == 0 else np.vstack((res, row))

df = pd.DataFrame(res, columns = ['price', 'liq'])
df.sort_values(by=['price'], inplace=True)
df.reset_index(drop=True,inplace=True)
df['price'] = np.round(df['price'].values, 3)
freq_df_v3 = df.groupby('price').agg({'liq': 'sum'})

In [ ]:
fig, (price_ax, ld_ax) = plt.subplots(nrows=2, sharex=False, sharey=False, figsize=(12, 10))

price_ax.plot(x_val[1:], p_arr[1:], color = 'r',linestyle = 'dashdot', label='initial invest') 
price_ax.set_title(f'Price Chart (ETH/TKN)', fontsize=15)
price_ax.set_ylabel('Price (USD)', size=14)
price_ax.set_xlabel('Date', size=10)

ld_ax.axvline(x = lp.get_price(tkn), color = 'red', linestyle = 'dashdot', linewidth=2, label = 'Last Price')
ld_ax.bar(freq_df_v2.index, freq_df_v2['liq'].values, color ='purple', width = 0.00095, alpha = .5, label = 'Uni V2')
ld_ax.bar(freq_df_v3.index, freq_df_v3['liq'].values, color ='teal', width = 0.00095, alpha = .5, label = 'Uni V3')
ld_ax.set_title('Liquidity Frequency Distribution: Uniswap V2 vs V3',fontsize=15)
ld_ax.set_ylabel('Liquidity', fontsize=14)
ld_ax.set_xlabel('Price (USD)', fontsize=10)
ld_ax.legend()